## Pipeline Workflow

This noteboook includes the workflow for both processes:
1. Data Pipeline
2. Model Pipeline

TODO:
- @All align on the python files/naming conventions and each steps
- @All make sure the steps reflect the one Fred proposed

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/Shareddrives/SIADS - 694-695 Team Drive/python-files"
!ls -l

/content/drive/Shareddrives/SIADS - 694-695 Team Drive/python-files
total 67
-rw------- 1 root root  1655 May 13 02:13 clean_dataset.py
-rw------- 1 root root  2434 May 12 00:19 create_sample_dataset.py
-rw------- 1 root root  3327 May 11 22:09 data_extraction.py
-rw------- 1 root root 17254 May 12 20:26 data_prep.py
-rw------- 1 root root     0 Apr 23 23:30 data_preprocessing.py
-rw------- 1 root root   165 May  3 22:13 ml_algo.py
-rw------- 1 root root 21978 May 13 02:16 orchestrator.ipynb
drwx------ 2 root root  4096 May 11 04:15 __pycache__
-rw------- 1 root root 10931 May 12 17:51 transaction_extraction.py
-rw------- 1 root root  4130 May 12 00:15 utils.py


In [ ]:
!python --version

Python 3.7.13


If you need a library that's not built into Colab's environment, you can add them as follows:

In [ ]:
## installing relevant packages
# !pip install google-search-results package2 package3 ...
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!pip install markupsafe~=2.1.1
!pip install folium==0.2.1
!pip install markupsafe==2.0.1
!pip install imbalanced-learn
!pip install scikit-learn-extra
!pip install factor_analyzer
!pip install --upgrade category_encoders

     \ 21.8 MB 148 kB/s
     |████████████████████████████████| 10.9 MB 8.6 MB/s 
     |████████████████████████████████| 596 kB 52.0 MB/s 
     |████████████████████████████████| 102 kB 39.4 MB/s 
     |████████████████████████████████| 690 kB 44.6 MB/s 
     |████████████████████████████████| 4.7 MB 39.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 812 kB 62.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.2.0-py2.py3-none-any.whl size=262599 sha256=085241c35cdd6a04a61ddcf8b992cf35e9157b84ce1612bace67cd3a45eefe8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-sh74457k/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=2e6f25ea7075d9fac762b3ce73d0442cc36a2dc52c1282e96b5782c86fd977fe
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14

### Step 1: Data Extraction Portion

Goal for this step: 
- Pull from BigQuery and generate 6 months frozen dataset

TODO: 
- [x] @Tia to refactor the extraction to reflect the notebook: `Data_Processing.ipynb`
- [Optional] @Tia refine dates logic to be customizable
- [x] @Tia add argparse


In [ ]:
!python data_extraction.py \
  --output_directory ../datasets/monthly-partitioned-data/ \
  --credentials_json ../credentials/compact-scene-317315-56e479e9e148.json

### Step 2: Transaction Extraction Portion

**WARNING: THIS TAKES ~12 HOURS TO RUN**

Goal:
- Pull all Transactions from monthly datasets from Step 1
- Pull all Non-Transactions from monthly datasets from Step 1

TODO: 
- [x] @Idris to refactor the extraction to reflect the notebook and pull all transactions from each monthly dataset: `Extraction_Data_Sample.ipynb`
- [x] @Idris add argparse


In [ ]:
%%time
!python transaction_extraction.py \
  --input_directory ../datasets/monthly_partitioned_data/ \
  --output_directory ../datasets/monthly_partitioned_data_transactions/

### Step 3: Data Sampling Portion

Goal:
1. Reads all transactions CSV from the input directory and join as one DF.
2. Reads all non-transactions CSV from the input directory and samples 10%
3. Join both results from Step 1 and 2 into one DF called `sample_dataset.csv`

TODO: 
- [x] @Idris to refactor the extraction to reflect the notebook: `Data_Processing.ipynb`
- [x] @Idris add argparse


In [ ]:
%%time
!python create_sample_dataset.py \
  --input_directory ../datasets/monthly_partitioned_data_transactions/ \
  --output_file ../datasets/sample_dataset.csv

../datasets/monthly_partitioned_data_transactions/ ../datasets/sample_dataset.csv
['../datasets/monthly_partitioned_data_transactions/non_transactions/non_transactions_from_January 2017 Google Analytics Dataset.csv', '../datasets/monthly_partitioned_data_transactions/non_transactions/non_transactions_from_August 2016 Google Analytics Dataset.csv', '../datasets/monthly_partitioned_data_transactions/non_transactions/non_transactions_from_September 2016 Google Analytics Dataset.csv', '../datasets/monthly_partitioned_data_transactions/non_transactions/non_transactions_from_October 2016 Google Analytics Dataset.csv', '../datasets/monthly_partitioned_data_transactions/non_transactions/non_transactions_from_November 2016 Google Analytics Dataset.csv', '../datasets/monthly_partitioned_data_transactions/non_transactions/non_transactions_from_December 2016 Google Analytics Dataset.csv']
100% 6/6 [03:32<00:00, 35.37s/it]
CPU times: user 2.91 s, sys: 595 ms, total: 3.5 s
Wall time: 6min 2s


### Step 4: Common Data Cleaning & Descriptive Analysis Portion

Goal:
- Drop unneeded columns (Refer to this [link](https://docs.google.com/spreadsheets/d/1fT-iZyGZnpkli9ve9EY9TRTfycNlIoxp/edit?usp=sharing&ouid=116636835356831800242&rtpof=true&sd=true))
- Cast to appropriate datatypes

On a different notebook:
- Visualizations
- EDA
- Describing Data
- Histograms
- Dashboarding ([Pandas Profiling](https://github.com/ydataai/pandas-profiling))

Assigned: 
- [x] @Tia


In [ ]:
!python clean_dataset.py \
  --input_file ../datasets/sample_dataset.csv \
  --output_file ../datasets/cleaned_dataset.csv \
  --dashboard_file ../datasets/data_dashboard.html

tcmalloc: large alloc 1073741824 bytes == 0x4654c000 @  0x7f76673da2a4 0x7f765582e9a5 0x7f765582fcc1 0x7f765583169e 0x7f765580250c 0x7f765580f399 0x7f76557f797a 0x59afff 0x515655 0x549e0e 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x4bcb19 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e
tcmalloc: large alloc 2147483648 bytes == 0x96d5e000 @  0x7f76673da2a4 0x7f765582e9a5 0x7f765582fcc1 0x7f765583169e 0x7f765580250c 0x7f765580f399 0x7f76557f797a 0x59afff 0x515655 0x549e0e 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x4bcb19 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x51566f 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e
Summarize dataset: 100% 101/101 [00:30<00:00,  3.31it/s, Completed]
Generate report structure: 100% 1/1 [00:06<00:00,  6.35s/it]
Render HTML: 100% 1/1 [00:04<00:00,  4.24s/it]
Export report to file: 100% 1/1 [00:00<00:00,  1.28it/s]


### Step 5: Data Prep for Modeling

Goal:
- Create 4 datasets as inputs for 4 models
- Prep should include:
  * Column Dropping
  * Column Encoding

The 4 datasets for each model should be:
- A1- Likelyhood to convert data set (Visitors) (Assigned: @Fred)
- B2- Complex clustering data set (Visitors) (Assigned: @Fred)
- B1- RFM data set (Assigned: @Fred)
- A2- Returning customers data set (Assigned: @Idris)
- [Optional] A3- Attribution model data set (Assigned: @Tia)


In [ ]:
!python data_prep.py \
  --input_file ../datasets/cleaned_dataset.csv \
  --output_directory ../datasets/model_files

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
../datasets/cleaned_dataset.csv ../datasets/model_files
2016-08-01 00:00:00 succesfully transformed to a datime object
Divided by 10^6
Transformations being made
1 date NO CHANGE
2 fullVisitorId NO CHANGE
3 socialEngagementType fillnan
3 socialEngagementType binary
3 socialEngagementType int64
4 channelGrouping one_hot drop
5 totals.hits int64
5 totals.hits NO CHANGE
6 totals.pageviews fillnan
7 totals.timeOnSite fillnan
8 totals.transactions fillnan
9 totals.newVisits fillnan
9 totals.newVisits int64
10 hits.eCommerceAction.action_type int64
10 hits.eCommerceAction.action_type NO CHANGE
11 totals.bounces fillnan
11 totals.bounces int64
0.1183562197092084
12 geoNetwork.country geoNetwork.country_woe woe
12 geoNetwork.country drop
0.1183562197092084
13 trafficSource.sou

In [ ]:
import pandas as pd

# df = pd.read_csv("../datasets/model_files/A1_B2_data.csv")
df = pd.read_csv("../datasets/model_files/changes_made.csv")

# Modeling Pipeline

Goal:
- @All test out our input dataset against our models and discuss results